In [7]:
import pandas as pd

df = pd.read_csv("./crawling/naver_blog_reviews_removed_final.csv")
print(df.head())
print(df.columns)

  search_query                                              title  \
0    도산근린공원 후기                 도산공원 강남 공원 서울 도심 숨겨진 명소 도산근린공원을 가다   
1    도산근린공원 후기               압구정 도산공원 산책로 서울 역사탐방 도산 안창호 기념관 관람후기   
2    도산근린공원 후기  [서울 강남구] 걸음은 가볍지만 마음은 무거웠던 도산 공원 산보 후기 (feat. ...   
3    도산근린공원 후기                            압구정 도산공원 처음 방문하여 산책한 후기   
4    도산근린공원 리뷰                                압구정, 어디까지 가봤니? 도산공원   

                                             content                 date  \
0  도산근린공원\r\n서울 도심의 숨겨진 명소\r\n도산공원 나들이\r\n복잡한 서울 ...   2023. 4. 14. 18:09   
1  도산안창호선생기념사업회\r\n도산 안창호 기념관\r\n도산근린공원\r\n✔️ 관람장...    2024. 3. 8. 19:35   
2  안녕하세요.\r\n호림 박물관 신사 분관을\r\n관람하고 나서니\r\n도산공원 비석...        2023. 12. 19.   
3  방문 계기 및 위치\r\n50m\r\n© NAVER Corp.\r\n도산공원\r\n...   2023. 10. 3. 23:29   
4  안녕하세요! 공원 고인물 존입니다\r\n오늘은 압구정 로데오거리 근처에 있는 산책하...  2021. 11. 27. 20:45   

                                            blog_url park_name  
0  https://blog.naver.com/homilbbangbooboo/223074...    도

In [8]:
# 결측치, 중복 제거
df = df.dropna(subset=["content"])
df = df.drop_duplicates(subset=["content"])

In [9]:
# 정규표현식으로 광고성 문구 제거
import re

ad_patterns = [
    r"협찬", r"광고", r"서포터즈", r"제공.?받아", r"이웃추가", 
    r"좋아요", r"더보기", r"클릭", r"후기입니다"
]

pattern = "|".join(ad_patterns)
df["content"] = df["content"].apply(lambda x: re.sub(pattern, "", str(x)))

In [10]:
# HTML 태그, 특수문자, 불필요한 공백 제거
from bs4 import BeautifulSoup

def clean_text(text):
    # HTML 태그 제거
    text = BeautifulSoup(text, "html.parser").get_text()
    # 이모지 제거
    text = re.sub(r"[^\uAC00-\uD7A3\u3130-\u318F\s.,!?]", " ", text)
    # 다중 공백 제거
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["content"] = df["content"].apply(clean_text)

In [11]:
# 좋아요가 들어가지만 긍정의 의미가 아닌
# 금지어/긍정오류 방지 처리
ban_phrases = ["좋아요 버튼", "구독", "팔로우", "이벤트 참여"]
for phrase in ban_phrases:
    df["content"] = df["content"].str.replace(phrase, "", regex=False)

In [12]:
# 문장 분리 -> 분석 대상 문장만 남기기
import kss  # 한국어 문장분리기

sentences = []
for review in df["content"]:
    for sent in kss.split_sentences(review):
        if len(sent) >= 5:  # 5글자 이상만
            sentences.append(sent)
df_sent = pd.DataFrame(sentences, columns=["sentence"])

[Kss]: Because there's no supported C++ morpheme analyzer, Kss will take pecab as a backend. :D
For your information, Kss also supports mecab backend.
We recommend you to install mecab or konlpy.tag.Mecab for faster execution of Kss.
Please refer to following web sites for details:
- mecab: https://github.com/hyunwoongko/python-mecab-kor
- konlpy.tag.Mecab: https://konlpy.org/en/latest/api/konlpy.tag/#mecab-class



KeyboardInterrupt: 

In [13]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 9.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.2/582.2 kB 7.7 MB/s eta 0:00:00


In [ ]:
# 형태소 분석, 불용어 제거
from konlpy.tag import Okt

okt = Okt()
stopwords = ["이", "그", "저", "그리고", "하지만", "또한"]

def tokenize(text):
    tokens = okt.morphs(text, stem=True)
    tokens = [t for t in tokens if t not in stopwords]
    return " ".join(tokens)

df_sent["tokens"] = df_sent["sentence"].apply(tokenize)

In [ ]:
# 전처리된 데이터 저장
df_sent.to_csv("test_cleaned.csv", index=False)